# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy as db

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle


In [5]:
# load data from database
engine = db.create_engine('sqlite:///DisasterDB.db')
connection = engine.connect()
df=pd.read_sql_table('message_map', connection)



X = df['message']
Y = df.iloc[:, 4:]


In [6]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25984,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25985,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25986,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25987,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [10]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
# Split into Train and Test
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state = 42)

In [12]:
# Pipeline using MultiOutputClassifier with a base model of KneighborsClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', MultiOutputClassifier(KNeighborsClassifier()))
                    ])


pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)

# Print the Precision, Recall and F1 score by category and overall precision, recall
for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.57      0.37      0.45      1563
           1       0.82      0.91      0.86      4935

    accuracy                           0.78      6498
   macro avg       0.70      0.64      0.66      6498
weighted avg       0.76      0.78      0.76      6498

request
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      5362
           1       0.66      0.59      0.62      1136

    accuracy                           0.87      6498
   macro avg       0.79      0.76      0.77      6498
weighted avg       0.87      0.87      0.87      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.50      0.03      0.06        34

    accuracy                           0.99      6498
   macro avg       0.75      0.51      0.53      6498
weighted avg       0.99      0.99      0.99      649


fire
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6442
           1       0.31      0.07      0.12        56

    accuracy                           0.99      6498
   macro avg       0.65      0.54      0.56      6498
weighted avg       0.99      0.99      0.99      6498

earthquake
              precision    recall  f1-score   support

           0       0.95      0.95      0.95      5883
           1       0.51      0.51      0.51       615

    accuracy                           0.91      6498
   macro avg       0.73      0.73      0.73      6498
weighted avg       0.91      0.91      0.91      6498

cold
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6379
           1       0.32      0.13      0.19       119

    accuracy                           0.98      6498
   macro avg       0.65      0.56      0.59      6498
weighted avg       0.97      0.98      0.97      649

In [18]:
# Optimize model by choosing best base model using Gridsearch to optimize for Recall
parameters = {
    'model__estimator': (KNeighborsClassifier(), RandomForestClassifier(), LogisticRegression())
}

In [22]:
# Grid search to find optimal base model, based on Recall parameter
cv = GridSearchCV(pipeline, param_grid = parameters, scoring='recall_micro')

In [23]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000023A6DCA45E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('model',
                                        MultiOutputClassifier(estimator=KNeighborsClassifier()))]),
             param_grid={'model__estimator': (KNeighborsClassifier(),
                                              RandomForestClassifier(),
                                              LogisticRegression())},
             scoring='recall_micro')

In [24]:
# KNeighborsClassifier is the best base model
cv.best_params_

{'model__estimator': KNeighborsClassifier()}

### 6. Improve your model
Use grid search to find better parameters. 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [38]:
# Optimize for hyperparameters using gridsearch
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (0.005, 0.01),
    'vect__ngram_range': ((1,1), (1,2)),
    'model__estimator__p': (1,2),
    'model__estimator__weights': ('uniform', 'distance')
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='recall_micro')

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [69]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize, max_df=1.0, ngram_range=(1,1))),
                      ('tfidf', TfidfTransformer()),
                      ('model', MultiOutputClassifier(KNeighborsClassifier(weights='distance', p=2)))
                    ])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.57      0.37      0.45      1563
           1       0.82      0.91      0.86      4935

    accuracy                           0.78      6498
   macro avg       0.70      0.64      0.66      6498
weighted avg       0.76      0.78      0.76      6498

request
              precision    recall  f1-score   support

           0       0.91      0.94      0.92      5362
           1       0.66      0.58      0.62      1136

    accuracy                           0.87      6498
   macro avg       0.79      0.76      0.77      6498
weighted avg       0.87      0.87      0.87      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.50      0.03      0.06        34

    accuracy                           0.99      6498
   macro avg       0.75      0.51      0.53      6498
weighted avg       0.99      0.99      0.99      649


direct_report
              precision    recall  f1-score   support

           0       0.89      0.92      0.90      5227
           1       0.62      0.52      0.56      1271

    accuracy                           0.84      6498
   macro avg       0.75      0.72      0.73      6498
weighted avg       0.83      0.84      0.84      6498

Overall Precision Score: 0.5514226365876045
 Overall Recall Score: 0.5498554913294798
 Overall Accuracy Score: 0.2020621729763004


Final K Neighbors Classifier Model is producing a Precision of 0.55 and a recall of 0.55. Let us see if we can improve some more on the model

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

We saw earlier that there was a hierarchical correlation between the predicted output classes. For example, when related = 0 all the other columns are = 0. When related = 1, one or more of the other columns are activated. There is also a correlation between the other columns. We therefore attempt ClassifierChain which takes previous predictions in the chain as features in addition to the original list of features in the model

In [47]:
# Build Classifier Chain Model Pipeline
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(KNeighborsClassifier()))
                    ])

In [52]:
# Grid Search for optimal base model - KNeighborsClassifier, RandomForestClassifier and LogisticRegression
parameters = {
    'model__base_estimator': (KNeighborsClassifier(), RandomForestClassifier(), LogisticRegression(max_iter=200))
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='recall_micro')

In [53]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000023A6DCA45E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('model',
                                        ClassifierChain(base_estimator=KNeighborsClassifier()))]),
             param_grid={'model__base_estimator': (KNeighborsClassifier(),
                                                   RandomForestClassifier(),
                                                   LogisticRegression(max_iter=200))},
             scoring='recall_micro')

In [54]:
# Grid search suggests that KNeighborsClassifier is the best base model for ClassifierChain as well
cv.best_params_

{'model__base_estimator': KNeighborsClassifier()}

In [55]:
# Pipeline with KneighborsClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(KNeighborsClassifier()))
                    ])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.57      0.37      0.45      1563
           1       0.82      0.91      0.86      4935

    accuracy                           0.78      6498
   macro avg       0.70      0.64      0.66      6498
weighted avg       0.76      0.78      0.76      6498

request
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      5362
           1       0.60      0.65      0.62      1136

    accuracy                           0.86      6498
   macro avg       0.76      0.78      0.77      6498
weighted avg       0.87      0.86      0.86      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.50      0.03      0.06        34

    accuracy                           0.99      6498
   macro avg       0.75      0.51      0.53      6498
weighted avg       0.99      0.99      0.99      649

C:\Users\subha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


tools
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6456
           1       0.00      0.00      0.00        42

    accuracy                           0.99      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      0.99      0.99      6498

hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6417
           1       0.33      0.01      0.02        81

    accuracy                           0.99      6498
   macro avg       0.66      0.51      0.51      6498
weighted avg       0.98      0.99      0.98      6498

shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6468
           1       0.00      0.00      0.00        30

    accuracy                           1.00      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      1.00      0.99      649

In [56]:
# Pipeline with RandomForestClassifier
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(RandomForestClassifier()))
                    ])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.76      0.27      0.39      1563
           1       0.81      0.97      0.88      4935

    accuracy                           0.80      6498
   macro avg       0.78      0.62      0.64      6498
weighted avg       0.79      0.80      0.76      6498

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5362
           1       0.88      0.45      0.60      1136

    accuracy                           0.89      6498
   macro avg       0.89      0.72      0.77      6498
weighted avg       0.89      0.89      0.88      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      0.99      0.99      649

C:\Users\subha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



shelter
              precision    recall  f1-score   support

           0       0.93      1.00      0.96      5908
           1       0.89      0.24      0.38       590

    accuracy                           0.93      6498
   macro avg       0.91      0.62      0.67      6498
weighted avg       0.93      0.93      0.91      6498

clothing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6392
           1       0.67      0.04      0.07       106

    accuracy                           0.98      6498
   macro avg       0.83      0.52      0.53      6498
weighted avg       0.98      0.98      0.98      6498

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6342
           1       0.86      0.04      0.07       156

    accuracy                           0.98      6498
   macro avg       0.92      0.52      0.53      6498
weighted avg       0.97      0.98      0.97      6

In [57]:
# Pipeline with LogisticRegression model
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(LogisticRegression(max_iter=200)))
                    ])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.75      0.40      0.52      1563
           1       0.83      0.96      0.89      4935

    accuracy                           0.82      6498
   macro avg       0.79      0.68      0.71      6498
weighted avg       0.81      0.82      0.80      6498

request
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      5362
           1       0.77      0.61      0.68      1136

    accuracy                           0.90      6498
   macro avg       0.85      0.79      0.81      6498
weighted avg       0.89      0.90      0.90      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      0.99      0.99      649

C:\Users\subha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6425
           1       0.00      0.00      0.00        73

    accuracy                           0.99      6498
   macro avg       0.49      0.50      0.50      6498
weighted avg       0.98      0.99      0.98      6498

refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6263
           1       0.65      0.06      0.10       235

    accuracy                           0.96      6498
   macro avg       0.81      0.53      0.54      6498
weighted avg       0.95      0.96      0.95      6498

death
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6194
           1       0.78      0.31      0.45       304

    accuracy                           0.96      6498
   macro avg       0.87      0.65      0.71      6498
weighted avg       0.96      0.96      0.96 

#### K Neighbors Classifier:
Precision - 0.48
Recall - 0.58

#### Radom Forest Classifier:
Precision - 0.82
Recall - 0.49

#### Logistic Regression:
Precision - 0.77
Recall - 0.58

Clearly, Logistic Regression is the most superior in terms of Recall which is what we are optimizing for. It also has higher precision than K neighbors Classifier. So we go with Logistic Regression model and see if we can further optimize

In [58]:
# We observe the correlation between the Y variables
corr = y_train.corr()

In [60]:
# We sort the Y variables by decreasing order of correlation and use that information to build a new model with 
# sequence of predictions such that prior column predictions are used as features in addition to model features
corr_sorted = []
for col in corr.loc['related'].sort_values(ascending=False).index:
    corr_sorted.append(y_train.columns.get_loc(col))

In [62]:
# Pipeline to optimize the classifierchain model with Logistic Regression and specified order of prediction

pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(LogisticRegression(max_iter=200), order=corr_sorted))
                    ])

pipeline.fit(X_train, y_train)


y_pred = pipeline.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.75      0.40      0.52      1563
           1       0.83      0.96      0.89      4935

    accuracy                           0.82      6498
   macro avg       0.79      0.68      0.71      6498
weighted avg       0.81      0.82      0.80      6498

request
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      5362
           1       0.73      0.65      0.69      1136

    accuracy                           0.90      6498
   macro avg       0.83      0.80      0.81      6498
weighted avg       0.89      0.90      0.89      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      0.99      0.99      649

C:\Users\subha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



refugees
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6263
           1       0.67      0.08      0.14       235

    accuracy                           0.97      6498
   macro avg       0.82      0.54      0.56      6498
weighted avg       0.96      0.97      0.95      6498

death
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      6194
           1       0.78      0.35      0.48       304

    accuracy                           0.96      6498
   macro avg       0.88      0.67      0.73      6498
weighted avg       0.96      0.96      0.96      6498

other_aid
              precision    recall  f1-score   support

           0       0.89      0.96      0.92      5604
           1       0.51      0.28      0.36       894

    accuracy                           0.86      6498
   macro avg       0.70      0.62      0.64      6498
weighted avg       0.84      0.86      0.85     

#### Logistic Regression with column ordering:
Precision - 0.76
Recall - 0.61

Clearly, the column ordering has improved the recall of the model and is superior to prior models

In [63]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('model', ClassifierChain(LogisticRegression(max_iter=200), order=corr_sorted))
                    ])



In [65]:
# Optimize using gridsearch for class weight and solver used in the base Logistic Regression model
parameters = {
    'model__base_estimator__class_weight': (None,'balanced'),
    'model__base_estimator__solver': ('lbfgs','liblinear')
}

cv = GridSearchCV(pipeline, param_grid = parameters, scoring='accuracy')
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x0000023A6DCA45E0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('model',
                                        ClassifierChain(base_estimator=LogisticRegression(max_iter=200),
                                                        order=[0, 3, 27, 34, 1,
                                                               17, 10, 31, 29,
                                                               11, 28, 4, 18, 9,
                                                               33, 20, 5, 19,
                                                               16, 26, 8, 15, 6,
                                                               13, 32, 21, 7,
                                                               12, 30, 14, ...]))]),
             param_grid={'model__base_estimator__class_

In [68]:
cv.best_params_

{'model__base_estimator__class_weight': None,
 'model__base_estimator__solver': 'lbfgs'}

In [67]:
y_pred = cv.predict(X_test)


for i, col in enumerate(y_test.columns):
    print("{}\n{}".format(col, classification_report(y_test[col], y_pred[:,i])))

pr_score = precision_score(y_test, y_pred, average='micro')
re_score = recall_score(y_test, y_pred, average='micro')
acc_score = accuracy_score(y_test, y_pred)
print("Overall Precision Score: {}\n Overall Recall Score: {}\n Overall Accuracy Score: {}". format(pr_score, re_score, 
                                                                                                    acc_score))

related
              precision    recall  f1-score   support

           0       0.75      0.40      0.52      1563
           1       0.83      0.96      0.89      4935

    accuracy                           0.82      6498
   macro avg       0.79      0.68      0.71      6498
weighted avg       0.81      0.82      0.80      6498

request
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      5362
           1       0.73      0.65      0.69      1136

    accuracy                           0.90      6498
   macro avg       0.83      0.80      0.81      6498
weighted avg       0.89      0.90      0.89      6498

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6464
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6498
   macro avg       0.50      0.50      0.50      6498
weighted avg       0.99      0.99      0.99      649

C:\Users\subha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


clothing
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6392
           1       0.71      0.11      0.20       106

    accuracy                           0.98      6498
   macro avg       0.85      0.56      0.59      6498
weighted avg       0.98      0.98      0.98      6498

money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6342
           1       0.77      0.13      0.22       156

    accuracy                           0.98      6498
   macro avg       0.87      0.56      0.60      6498
weighted avg       0.97      0.98      0.97      6498

missing_people
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6425
           1       0.00      0.00      0.00        73

    accuracy                           0.99      6498
   macro avg       0.49      0.50      0.50      6498
weighted avg       0.98      0.99      0.98 

### 9. Export your model as a pickle file

In [70]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(pipeline, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.